# Carga de Documentos: Fundamentos de RAG

## Introducción a la Recuperación y Generación Aumentada

En este cuaderno aprenderemos los fundamentos de RAG (Retrieval Augmented Generation) comenzando por el primer paso crítico: **cargar y procesar documentos**.

RAG es una técnica que permite a los modelos de lenguaje acceder a información específica de documentos externos durante la generación de respuestas, lo que resulta especialmente útil para trabajar con datos empresariales, técnicos o especializados.

## ¿Qué es la Recuperación y Generación Aumentada (RAG)?

En RAG (Retrieval Augmented Generation), un modelo de lenguaje grande (LLM) recupera documentos contextuales de un conjunto de datos externo como parte de su proceso de ejecución.

**¿Por qué es útil RAG?**

RAG es especialmente útil cuando queremos hacer preguntas sobre documentos específicos como:
- PDFs de políticas empresariales
- Manuales técnicos o procedimientos
- Bases de conocimiento internas
- Documentación especializada
- Videos educativos o de capacitación

**¿Cómo funciona?**

1. **Cargar documentos** (lo que aprenderemos hoy)
2. **Dividir en fragmentos** para procesamiento eficiente
3. **Crear embeddings** (representaciones vectoriales)
4. **Buscar información relevante** cuando llega una consulta
5. **Generar respuesta** basada en los documentos encontrados

### Diagrama del Proceso RAG

El siguiente diagrama muestra cómo funciona el proceso completo de RAG:

![Proceso RAG](attachment:overview.jpeg)

Como se observa, el primer paso fundamental es la **carga de documentos**, que es exactamente lo que aprenderemos en este cuaderno.

In [ ]:
# Instalación de librerías necesarias
# LangChain es la librería principal para construir aplicaciones RAG
!pip install langchain -q

print("Instalación completada")

Instalación completada


In [ ]:
# Configuración del entorno
# Detectamos si estamos en Google Colab o en un entorno local
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
    print("Entorno detectado: Google Colab")
except ImportError:
    IN_COLAB = False
    print("Entorno detectado: Local")

import os
import sys

# Configuración de OpenAI (opcional para este cuaderno)
if IN_COLAB:
    try:
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        print("Clave API cargada desde secretos de Colab")
    except:
        print("No se encontró OPENAI_API_KEY en secretos de Colab")
        OPENAI_API_KEY = None
else:
    # En local: usar variable de entorno o archivo .env
    try:
        from dotenv import load_dotenv, find_dotenv
        load_dotenv(find_dotenv())
        OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
        print("Variables de entorno cargadas")
    except ImportError:
        print("dotenv no instalado. Usando variables de entorno del sistema")
        OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

if OPENAI_API_KEY:
    import openai
    openai.api_key = OPENAI_API_KEY
    print("OpenAI configurado correctamente")
else:
    print("OpenAI no configurado (opcional para este cuaderno)")

Entorno detectado: Google Colab
Clave API cargada desde secretos de Colab
OpenAI configurado correctamente


## Carga de Documentos PDF

Los archivos PDF son una de las fuentes más comunes de información empresarial y técnica. LangChain proporciona herramientas especializadas para extraer texto de PDFs y convertirlos en un formato que podemos usar para RAG.

**Ejemplo práctico:** Vamos a cargar un PDF de una conferencia de machine learning de Andrew Ng de la Universidad de Stanford. Estos documentos son transcripciones automáticas, por lo que a veces las palabras y oraciones pueden estar divididas de manera inesperada.

**Aplicaciones reales:**
- Manuales de políticas empresariales
- Documentos técnicos y especificaciones
- Reportes e informes
- Material educativo y de capacitación

In [ ]:
# Instalamos la librería para trabajar con PDFs
# PyPDF2 es la librería más popular para extraer texto de archivos PDF
!pip install pypdf -q

print("PyPDF instalado correctamente")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 3.9 MB/s eta 0:00:00
PyPDF instalado correctamente


In [ ]:
!pip install -U langchain-community -q
print("langchain-community instalado correctamente")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain-community instalado correctamente


In [ ]:
# Cargamos un PDF usando PyPDFLoader de LangChain
from langchain.document_loaders import PyPDFLoader

# Especificamos la ruta del archivo PDF
# En un caso real, aquí pondrías la ruta a tu propio documento
loader = PyPDFLoader("/content/Generación de imágenes.pdf")

# Cargamos todas las páginas del PDF
pages = loader.load()

print(f"PDF cargado exitosamente: {len(pages)} páginas")
print("Cada página se convierte en un objeto Document independiente")

PDF cargado exitosamente: 20 páginas
Cada página se convierte en un objeto Document independiente


### Estructura de un Document en LangChain

Cada página del PDF se convierte en un objeto `Document` que contiene dos componentes principales:

1. **`page_content`**: El texto extraído de la página
2. **`metadata`**: Información adicional como número de página, fuente, etc.

Esta estructura estándar permite que LangChain procese documentos de diferentes fuentes (PDF, web, videos, etc.) de manera uniforme.

In [ ]:
# Verificamos cuántas páginas se cargaron
len(pages)

20

In [ ]:
# Examinamos la primera página como ejemplo
page = pages[0]
print("Primera página seleccionada para análisis")

Primera página seleccionada para análisis


In [ ]:
# Mostramos los primeros 500 caracteres del contenido de la página
print("CONTENIDO DE LA PRIMERA PÁGINA (primeros 500 caracteres):")
print("=" * 60)
print(page.page_content[0:500])
print("=" * 60)
print("(Contenido truncado para visualización)")

CONTENIDO DE LA PRIMERA PÁGINA (primeros 500 caracteres):
1 / 2 0 
Generación de imágenes
Se usó la API de Cloud Translation para traducir esta página.
Puedes generar imágenes con la API de Gemini con las funciones multimodales integradas de Gemini o con Imagen,
los modelos de generación de imágenes especializados de Google. Para la mayoría de los casos de uso, comienza
con Gemini. Elige Imagen para tareas especializadas en las que la calidad de la imagen es fundamental. Consulta la
sección Cómo elegir el modelo adecuado para obtener más orientación.
T
(Contenido truncado para visualización)


In [ ]:
# Examinamos los metadatos de la página
print("METADATOS DE LA PÁGINA:")
print("=" * 30)
print(page.metadata)
print("=" * 30)
print("Los metadatos incluyen información como número de página y archivo fuente")

METADATOS DE LA PÁGINA:
{'producer': 'Skia/PDF m107', 'creator': 'Chromium', 'creationdate': '2025-07-31T17:55:16+00:00', 'moddate': '2025-07-31T17:55:16+00:00', 'source': '/content/Generación de imágenes.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}
Los metadatos incluyen información como número de página y archivo fuente


## Carga de Contenido Web (URLs)

Las páginas web contienen información actualizada y dinámica que puede ser extremadamente valiosa para sistemas RAG. LangChain puede extraer y procesar contenido directamente desde URLs.

**Casos de uso comunes:**
- Documentación técnica online
- Políticas y procedimientos publicados en sitios web corporativos
- Artículos y recursos educativos
- Bases de conocimiento públicas

In [ ]:
# Importamos y configuramos el loader para páginas web
from langchain.document_loaders import WebBaseLoader

# Especificamos la URL que queremos procesar
# En este ejemplo usamos el repo de nuestro curso
url = "https://es.wikipedia.org/wiki/Agricultura_en_Argentina"
loader = WebBaseLoader(url)

print(f"WebBaseLoader configurado para: {url}")
print("Listo para extraer contenido web")

WebBaseLoader configurado para: https://es.wikipedia.org/wiki/Agricultura_en_Argentina
Listo para extraer contenido web


**Nota**: La URL utilizada en este ejemplo ha sido actualizada para 2024. En versiones anteriores del curso se usaba una URL diferente.

In [ ]:
# Cargamos el contenido de la página web
docs = loader.load()

print(f"Contenido web cargado exitosamente: {len(docs)} documento(s)")
print("El HTML ha sido procesado y convertido a texto plano")

Contenido web cargado exitosamente: 1 documento(s)
El HTML ha sido procesado y convertido a texto plano


In [ ]:
# Mostramos una muestra del contenido extraído de la web
print("CONTENIDO EXTRAÍDO DE LA WEB (primeros 1500 caracteres):")
print("=" * 60)
print(docs[0].page_content[:3000])
print("=" * 60)
print("El contenido web ha sido limpiado y estructurado para procesamiento")

CONTENIDO EXTRAÍDO DE LA WEB (primeros 1500 caracteres):




Agricultura en Argentina - Wikipedia, la enciclopedia libre
































Ir al contenido







Menú principal





Menú principal
mover a la barra lateral
ocultar



		Navegación
	


PortadaPortal de la comunidadActualidadCambios recientesPáginas nuevasPágina aleatoriaAyudaNotificar un errorPáginas especiales



















Buscar











Buscar






















Apariencia
















Donaciones

Crear una cuenta

Acceder








Herramientas personales





Donaciones Crear una cuenta Acceder





		Páginas para editores desconectados más información



ContribucionesDiscusión




























Contenidos
mover a la barra lateral
ocultar




Inicio





1
Historia




Alternar subsección Historia





1.1
Historia precolombina








1.2
Colonización española








1.3
Siglo XIX








1.4
Siglo XX








1.5
Siglo XXI








1.6
Concentración de la tierra








1.7
Sojiz

In [ ]:
print("METADATOS DE LA PRIMERA PÁGINA:")
print("=" * 30)
print(docs[0].metadata)
print("=" * 30)
print("Los metadatos incluyen información como número de página y archivo fuente")

METADATOS DE LA PRIMERA PÁGINA:
{'source': 'https://es.wikipedia.org/wiki/Agricultura_en_Argentina', 'title': 'Agricultura en Argentina - Wikipedia, la enciclopedia libre', 'language': 'es'}
Los metadatos incluyen información como número de página y archivo fuente


## Resumen y Próximos Pasos

### Lo que aprendimos en este cuaderno:

1. **Fundamentos de RAG**: Entendimos qué es la Recuperación y Generación Aumentada y por qué es importante
2. **Carga de PDFs**: Procesamos documentos PDF y extrajimos texto estructurado
3. **Procesamiento de YouTube**: Convertimos audio de videos en transcripciones de texto
4. **Contenido web**: Extrajimos información directamente desde URLs
5. **Integración con Notion**: Importamos documentación empresarial desde Notion

### Estructura unificada de Document:

Independientemente de la fuente (PDF, audio, web, Notion), todos los documentos se convierten al formato estándar de LangChain:
- **`page_content`**: El texto extraído
- **`metadata`**: Información contextual (página, fuente, etc.)

### Próximos pasos en RAG:

1. **Fragmentación**: Dividir documentos largos en chunks manejables
2. **Embeddings**: Convertir texto en representaciones vectoriales
3. **Almacenamiento vectorial**: Guardar en bases de datos especializadas
4. **Recuperación**: Buscar contenido relevante por similitud semántica
5. **Generación**: Combinar información recuperada con LLMs para respuestas precisas

### Aplicaciones prácticas:

Este cuaderno proporciona la base para construir sistemas que pueden:
- Responder preguntas sobre documentos empresariales específicos
- Crear asistentes de atención al cliente basados en manuales
- Desarrollar sistemas de búsqueda inteligente en bases de conocimiento
- Automatizar el análisis de grandes volúmenes de documentación técnica

**¡Estás listo para el siguiente paso en la construcción de sistemas RAG completos!**